In [14]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import time

In [2]:
# data preparation
df_train = pd.read_csv('data/training.csv')
df_test = pd.read_csv('data/testing.csv')
df = pd.read_csv('data/race-result-horse.csv')
m, n = np.shape(df)
m_train, _ = np.shape(df_train)
m_test, _ = np.shape(df_test)

#produce dicts for horse,jockey,trainer
horse=list()
jockey=list()
trainer=list()
for i in range(len(df)):
    if df.horse_name[i] not in horse:
        horse.append(df.horse_name[i])
    if df.jockey[i] not in jockey:
        jockey.append(df.jockey[i])
    if df.trainer[i] not in jockey:
        trainer.append(df.trainer[i])
train_horse=np.zeros((m_train,1))
train_jockey=np.zeros((m_train,1))
train_trainer=np.zeros((m_train,1))

In [3]:
# prepare training data
for i in range(m_train):
    train_horse[i,0] = horse.index(df_train.horse_name[i])
    train_jockey[i,0] = jockey.index(df_train.jockey[i])
    train_trainer[i,0] = trainer.index(df_train.trainer[i])
actual_weight = df_train.actual_weight.reshape((m_train,1))
declared_weight = df_train.declared_horse_weight.reshape((m_train,1))
draw = df_train.draw.reshape((m_train,1))
win_odds = df_train.win_odds.reshape((m_train,1))
race_distance = df_train.race_distance.reshape((m_train,1))

# we use horse, jockey, trainer, actual weight, declared weight, win odds, race distance as independent variables
X_train = np.hstack((train_horse, train_jockey, train_trainer, actual_weight,
                     declared_weight, draw, win_odds, race_distance)) 
y_train = df_train.finishing_position

# prepare test data
test_horse=np.zeros((m_test,1))
test_jockey=np.zeros((m_test,1))
test_trainer=np.zeros((m_test,1))

#print(df_test.horse_name[])
for i in range(len(df_test)):
    test_horse[i,0] = horse.index(df_test.horse_name[i])
    test_jockey[i,0] = jockey.index(df_test.jockey[i])
    test_trainer[i,0] = trainer.index(df_test.trainer[i])
actual_weight_test = df_test.actual_weight.reshape((m_test,1))
declared_weight_test = df_test.declared_horse_weight.reshape((m_test,1))
draw_test = df_test.draw.reshape((m_test,1))
win_odds_test = df_test.win_odds.reshape((m_test,1))
race_distance_test = df_test.race_distance.reshape((m_test,1))
X_test = np.hstack((test_horse,test_jockey,test_trainer,actual_weight_test,
                    declared_weight_test, draw_test, win_odds_test, race_distance_test))
y_test = df_test.finishing_position

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  import sys
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  # Remove the CWD from sys.path while we load

In [15]:
#3.1.1 Logistic 
start_time = time.time()
lr_model = LogisticRegression()
lr_model.fit(X_train,y_train)
lr_result = lr_model.predict(X_test)
print('training time: ', time.time()-start_time, 'secends')
# print(lr_result.shape)
lr_score = lr_model.score(X_test,y_test)
print(lr_score)

training time:  1.3422021865844727 secends
0.1309381917248425


In [16]:
# 3.1.2 Naïve Bayes
start_time = time.time()
nb_model = sklearn.naive_bayes.GaussianNB()
nb_model.fit(X_train, y_train)
nb_result=nb_model.predict(X_test)
print('training time: ', time.time()-start_time, 'secends')
nb_score = nb_model.score(X_test, y_test)
print(nb_score)


training time:  0.01826190948486328 secends
0.11901924059254214


In [17]:
# 3.1.3 SVM
from sklearn.svm import LinearSVC
#svm_model = SVC(kernel='linear')
#print('model')
#svm_model.fit(X_train, y_train)
#print('fit')
#svm_score = svm_model.score(X_test, y_test)
#print(svm_score)
start_time = time.time()
svm_model = LinearSVC(random_state=0)
#print('model')
svm_model.fit(X_train, y_train)
#print('fit')
svm_result=svm_model.predict(X_test)
print('training time: ', time.time()-start_time, 'secends')
svm_score = svm_model.score(X_test, y_test)
#print(svm_model.coef_)
#print(svm_model.predict(X_test))
print(svm_score)

training time:  26.902045965194702 secends
0.062148816618423294


In [19]:
#3.1.4 Random Forest
from sklearn.ensemble import RandomForestClassifier
start_time = time.time()
rf_model=RandomForestClassifier(max_depth=2,random_state=0)
rf_model.fit(X_train,y_train)
rf_result=rf_model.predict(X_test)
print('training time: ', time.time()-start_time, 'secends')
#print(rf_model.predict(X_test))
print(rf_model.score(X_test,y_test))

training time:  0.09040689468383789 secends
0.12821386003745955


In [8]:
#3.2
from collections import Counter
#The lr matrix
horse_win_lr=np.zeros((len(lr_result),1))
horse_top3_lr=np.zeros((len(lr_result),1))
horse_top50percent_lr=np.zeros((len(lr_result),1))
count_of_race_participation=Counter(df_test.race_id)
for i in range(len(lr_result)):
    if lr_result[i] == 1:
         horse_win_lr[i]=1
    if lr_result[i] <= 3:
         horse_top3_lr[i] = 1
    if lr_result[i]<= np.floor(count_of_race_participation[df_test.race_id[i]]/2):
         horse_top50percent_lr[i] = 1
headers=['RaceID','HorseID','HorseWin','HorseRankTop3','HorseRankTop50Percent']
import csv
with open('lr_predictions.csv','w') as f1:
     lr_csv=csv.writer(f1)
     lr_csv.writerow(headers)
     for i in range(len(lr_result)):
         lr_csv.writerow([df_test.race_id[i],df_test.horse_id[i],horse_win_lr[i][0],horse_top3_lr[i][0],horse_top50percent_lr[i][0]])
#the naive bayesian matrix
horse_win_nb=np.zeros((len(nb_result),1))
horse_top3_nb=np.zeros((len(nb_result),1))
horse_top50percent_nb=np.zeros((len(nb_result),1))
count_of_race_participation=Counter(df_test.race_id)
for i in range(len(nb_result)):
    if nb_result[i] == 1:
        horse_win_nb[i]=1
    if nb_result[i] <= 3:
        horse_top3_nb[i] = 1
    if nb_result[i]<= np.floor(count_of_race_participation[df_test.race_id[i]]/2):
        horse_top50percent_nb[i] = 1
with open('nb_predictions.csv','w') as f2:
     nb_csv=csv.writer(f2)
     nb_csv.writerow(headers)
     for i in range(len(nb_result)):
         nb_csv.writerow([df_test.race_id[i],df_test.horse_id[i],horse_win_nb[i][0],horse_top3_nb[i][0],horse_top50percent_nb[i][0]])
#the svm matrix
horse_win_svm=np.zeros((len(svm_result),1))
horse_top3_svm=np.zeros((len(svm_result),1))
horse_top50percent_svm=np.zeros((len(svm_result),1))
count_of_race_participation=Counter(df_test.race_id)
for i in range(len(svm_result)):
    if svm_result[i] == 1:
        horse_win_svm[i]=1
    if svm_result[i] <= 3:
        horse_top3_svm[i] = 1
    if svm_result[i]<= np.floor(count_of_race_participation[df_test.race_id[i]]/2):
        horse_top50percent_svm[i] = 1
with open('svm_predictions.csv','w') as f3:
    svm_csv=csv.writer(f3)
    svm_csv.writerow(headers)
    for i in range(len(svm_result)):
        svm_csv.writerow([df_test.race_id[i],df_test.horse_id[i],horse_win_svm[i][0],horse_top3_svm[i][0],horse_top50percent_svm[i][0]])
#the rf matrix
horse_win_rf=np.zeros((len(rf_result),1))
horse_top3_rf=np.zeros((len(rf_result),1))
horse_top50percent_rf=np.zeros((len(rf_result),1))
count_of_race_participation=Counter(df_test.race_id)
for i in range(len(rf_result)):
    if rf_result[i] == 1:
        horse_win_rf[i]=1
    if rf_result[i] <= 3:
        horse_top3_rf[i] = 1
    if rf_result[i]<= np.floor(count_of_race_participation[df_test.race_id[i]]/2):
        horse_top50percent_rf[i] = 1
with open('rf_predictions.csv','w') as f4:
    rf_csv=csv.writer(f4)
    rf_csv.writerow(headers)
    for i in range(len(rf_result)):
        rf_csv.writerow([df_test.race_id[i],df_test.horse_id[i],horse_win_rf[i][0],horse_top3_rf[i][0],horse_top50percent_rf[i][0]])

In [9]:
#3.3
#get actual results from y_test
horse_win_actual=np.zeros((len(y_test),1))
horse_top3_actual=np.zeros((len(y_test),1))
horse_top50percent_actual=np.zeros((len(y_test),1))
count_of_race_participation=Counter(df_test.race_id)
for i in range(len(y_test)):
    if y_test[i]==1:
         horse_win_actual[i]=1
    if y_test[i]<=3:
         horse_top3_actual[i]=1
    if y_test[i]<=np.floor(count_of_race_participation[df_test.race_id[i]]/2):
         horse_top50percent_actual[i]=1
#lr model
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#lr horse_win
    if (horse_win_lr[i]==1 and horse_win_actual[i]==1):
        TP=TP+1
    if (horse_win_lr[i]==1 and horse_win_actual[i]==0):
        FP=FP+1
    if (horse_win_lr[i]==0 and horse_win_actual[i]==1):
        FN=FN+1
    if (horse_win_lr[i]==0 and horse_win_actual[i]==0):
        TN=TN+1
print("Recall of logistic model horse_win prediction= ", TP/(TP+FN))
print("Precision of logistic model horse_win prediction= ", TP/(TP+FP))
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#lr_top3
    if (horse_top3_lr[i]==1 and horse_top3_actual[i]==1):
        TP=TP+1
    if (horse_top3_lr[i]==1 and horse_top3_actual[i]==0):
        FP=FP+1
    if (horse_top3_lr[i]==0 and horse_top3_actual[i]==1):
        FN=FN+1
    if (horse_top3_lr[i]==0 and horse_top3_actual[i]==0):
        TN=TN+1
print("Recall of logistic model horse_top3 prediction= ", TP/(TP+FN))
print("Precision of logistic model horse_top3 prediction= ", TP/(TP+FP))
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#lr_top50percent
    if (horse_top50percent_lr[i]==1 and horse_top50percent_actual[i]==1):
        TP=TP+1
    if (horse_top50percent_lr[i]==1 and horse_top50percent_actual[i]==0):
        FP=FP+1
    if (horse_top50percent_lr[i]==0 and horse_top50percent_actual[i]==1):
        FN=FN+1
    if (horse_top50percent_lr[i]==0 and horse_top50percent_actual[i]==0):
        TN=TN+1
print("Recall of logistic model horse_top50percent prediction= ", TP/(TP+FN))
print("Precision of logistic model horse_top50percent prediction= ", TP/(TP+FP))
#naive bayesian
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#nb horse_win
    if (horse_win_nb[i]==1 and horse_win_actual[i]==1):
        TP=TP+1
    if (horse_win_nb[i]==1 and horse_win_actual[i]==0):
        FP=FP+1
    if (horse_win_nb[i]==0 and horse_win_actual[i]==1):
        FN=FN+1
    if (horse_win_nb[i]==0 and horse_win_actual[i]==0):
        TN=TN+1
print("Recall of naive bayesian model horse_win prediction= ", TP/(TP+FN))
print("Precision of naive bayesian model horse_win prediction= ", TP/(TP+FP))
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#lr_top3
    if (horse_top3_nb[i]==1 and horse_top3_actual[i]==1):
        TP=TP+1
    if (horse_top3_nb[i]==1 and horse_top3_actual[i]==0):
        FP=FP+1
    if (horse_top3_nb[i]==0 and horse_top3_actual[i]==1):
        FN=FN+1
    if (horse_top3_nb[i]==0 and horse_top3_actual[i]==0):
        TN=TN+1
print("Recall of naive bayesian model horse_top3 prediction= ", TP/(TP+FN))
print("Precision of naive bayesian model horse_top3 prediction= ", TP/(TP+FP))
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#lr_top50percent
    if (horse_top50percent_nb[i]==1 and horse_top50percent_actual[i]==1):
        TP=TP+1
    if (horse_top50percent_nb[i]==1 and horse_top50percent_actual[i]==0):
        FP=FP+1
    if (horse_top50percent_nb[i]==0 and horse_top50percent_actual[i]==1):
        FN=FN+1
    if (horse_top50percent_nb[i]==0 and horse_top50percent_actual[i]==0):
        TN=TN+1
print("Recall of naive bayesian model horse_top50percent prediction= ", TP/(TP+FN))
print("Precision of naive bayesian model horse_top50percent prediction= ", TP/(TP+FP))
#svm
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#nb horse_win
    if (horse_win_svm[i]==1 and horse_win_actual[i]==1):
        TP=TP+1
    if (horse_win_svm[i]==1 and horse_win_actual[i]==0):
        FP=FP+1
    if (horse_win_svm[i]==0 and horse_win_actual[i]==1):
        FN=FN+1
    if (horse_win_svm[i]==0 and horse_win_actual[i]==0):
        TN=TN+1
if ((TP+FN)==0 or (TP+FP)==0):
    print("Recall of svm model horse_win prediction and Precision of svm model horse_win prediction= 0")
else:
    print("Recall of svm model horse_win prediction= ", TP/(TP+FN))
    print("Precision of svm model horse_win prediction= ", TP/(TP+FP))
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#lr_top3
    if (horse_top3_svm[i]==1 and horse_top3_actual[i]==1):
        TP=TP+1
    if (horse_top3_svm[i]==1 and horse_top3_actual[i]==0):
        FP=FP+1
    if (horse_top3_svm[i]==0 and horse_top3_actual[i]==1):
        FN=FN+1
    if (horse_top3_svm[i]==0 and horse_top3_actual[i]==0):
        TN=TN+1
if ((TP+FN)==0 or (TP+FP)==0):
     print("Recall of svm model horse_top3 prediction and Precision of svm model horse_top3 prediction= 0")
else:
     print("Recall of svm model horse_top3 prediction= ", TP/(TP+FN))
     print("Precision of svm model horse_top3 prediction= ", TP/(TP+FP))
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#lr_top50percent
    if (horse_top50percent_svm[i]==1 and horse_top50percent_actual[i]==1):
        TP=TP+1
    if (horse_top50percent_svm[i]==1 and horse_top50percent_actual[i]==0):
        FP=FP+1
    if (horse_top50percent_svm[i]==0 and horse_top50percent_actual[i]==1):
        FN=FN+1
    if (horse_top50percent_svm[i]==0 and horse_top50percent_actual[i]==0):
        TN=TN+1
if ((TP+FN)==0 or (TP+FP)==0):
     print("Recall of svm model horse_top50percent prediction and Precision of svm model horse_top50percent prediction= 0")
else:
     print("Recall of svm model horse_top50percent prediction= ", TP/(TP+FN))
     print("Precision of svm model horse_top50percent prediction= ", TP/(TP+FP))
#rf model
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#nb horse_win
    if (horse_win_rf[i]==1 and horse_win_actual[i]==1):
        TP=TP+1
    if (horse_win_rf[i]==1 and horse_win_actual[i]==0):
        FP=FP+1
    if (horse_win_rf[i]==0 and horse_win_actual[i]==1):
        FN=FN+1
    if (horse_win_rf[i]==0 and horse_win_actual[i]==0):
        TN=TN+1
print("Recall of rf model horse_win prediction= ", TP/(TP+FN))
print("Precision of rf model horse_win prediction= ", TP/(TP+FP))
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#lr_top3
    if (horse_top3_rf[i]==1 and horse_top3_actual[i]==1):
        TP=TP+1
    if (horse_top3_rf[i]==1 and horse_top3_actual[i]==0):
        FP=FP+1
    if (horse_top3_rf[i]==0 and horse_top3_actual[i]==1):
        FN=FN+1
    if (horse_top3_rf[i]==0 and horse_top3_actual[i]==0):
        TN=TN+1
print("Recall of rf model horse_top3 prediction= ", TP/(TP+FN))
print("Precision of rf model horse_top3 prediction= ", TP/(TP+FP))
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#lr_top50percent
    if (horse_top50percent_rf[i]==1 and horse_top50percent_actual[i]==1):
        TP=TP+1
    if (horse_top50percent_rf[i]==1 and horse_top50percent_actual[i]==0):
        FP=FP+1
    if (horse_top50percent_rf[i]==0 and horse_top50percent_actual[i]==1):
        FN=FN+1
    if (horse_top50percent_rf[i]==0 and horse_top50percent_actual[i]==0):
        TN=TN+1
print("Recall of rf model horse_top50percent prediction= ", TP/(TP+FN))
print("Precision of rf model horse_top50percent prediction= ", TP/(TP+FP))


Recall of logistic model horse_win prediction=  0.7531380753138075
Precision of logistic model horse_win prediction=  0.16666666666666666
Recall of logistic model horse_top3 prediction=  0.8141160027952481
Precision of logistic model horse_top3 prediction=  0.37042925278219396
Recall of logistic model horse_top50percent prediction=  0.8164754953076121
Precision of logistic model horse_top50percent prediction=  0.6247340425531915
Recall of naive bayesian model horse_win prediction=  0.5460251046025104
Precision of naive bayesian model horse_win prediction=  0.16131025957972805
Recall of naive bayesian model horse_top3 prediction=  0.7351502445842069
Precision of naive bayesian model horse_top3 prediction=  0.3349251830627189
Recall of naive bayesian model horse_top50percent prediction=  0.9037191518943344
Precision of naive bayesian model horse_top50percent prediction=  0.5877034358047016
Recall of svm model horse_win prediction and Precision of svm model horse_win prediction= 0
Recall 